In [0]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('Project').getOrCreate()

In [0]:
df = spark.read.csv('/FileStore/tables/dog_food.csv',inferSchema=True,header=True)

In [0]:
df.printSchema()

root
-- A: integer (nullable = true)
-- B: integer (nullable = true)
-- C: double (nullable = true)
-- D: integer (nullable = true)
-- Spoiled: double (nullable = true)

In [0]:
for item in df.head(1)[0]:
  print(item)

4
2
12.0
3
1.0

In [0]:
df.summary().show()

+-------+------------------+------------------+------------------+------------------+-------------------+
summary| A| B| C| D| Spoiled|
+-------+------------------+------------------+------------------+------------------+-------------------+
 count| 490| 490| 490| 490| 490|
 mean| 5.53469387755102| 5.504081632653061| 9.126530612244897| 5.579591836734694| 0.2857142857142857|
 stddev|2.9515204234399057|2.8537966089662063|2.0555451971054275|2.8548369309982857|0.45221563164613465|
 min| 1| 1| 5.0| 1| 0.0|
 25%| 3| 3| 8.0| 3| 0.0|
 50%| 5| 6| 9.0| 6| 0.0|
 75%| 8| 8| 11.0| 8| 1.0|
 max| 10| 10| 14.0| 10| 1.0|
+-------+------------------+------------------+------------------+------------------+-------------------+

In [0]:
from pyspark.sql import functions as F

from pyspark.sql.functions import isnan, when, count, col

df.select([count(when(isnan(c), c)).alias(c) for c in df.columns]).show()

+---+---+---+---+-------+
 A| B| C| D|Spoiled|
+---+---+---+---+-------+
 0| 0| 0| 0| 0|
+---+---+---+---+-------+

In [0]:
from pyspark.ml.feature import VectorAssembler, VectorIndexer

In [0]:
assembler = VectorAssembler(inputCols=['A'
                                       ,'B'
                                       ,'C'
                                       ,'D'
                                      ]
                           ,outputCol='features')

In [0]:
output = assembler.transform(df)

In [0]:
final_df = output.select('features','Spoiled')

In [0]:
train_df,test_df = final_df.randomSplit([0.7,0.3])

In [0]:
from pyspark.ml.classification import DecisionTreeClassifier,GBTClassifier,RandomForestClassifier

In [0]:
dtc = DecisionTreeClassifier(labelCol='Spoiled',featuresCol='features')
rfc = RandomForestClassifier(labelCol='Spoiled',featuresCol='features')
gbt = GBTClassifier(labelCol='Spoiled',featuresCol='features')

In [0]:
dtc_model = dtc.fit(train_df)
rfc_model = rfc.fit(train_df)
gbt_model = gbt.fit(train_df)

In [0]:
dtc_preds = dtc_model.transform(test_df)
rfc_preds = rfc_model.transform(test_df)
gbt_preds = gbt_model.transform(test_df)

In [0]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [0]:
binary_eval = BinaryClassificationEvaluator(labelCol='Spoiled')

In [0]:
print('DTC Accuracy')
print(binary_eval.evaluate(dtc_preds))

DTC Accuracy
0.981318111053451

In [0]:
print('RFC Accuracy')
print(binary_eval.evaluate(rfc_preds))

RFC Accuracy
0.9711987545407369

In [0]:
binary_eval_gbt = BinaryClassificationEvaluator(labelCol='Spoiled',rawPredictionCol='prediction')

In [0]:
print('GBT Correct Accuracy')
print(binary_eval_gbt.evaluate(gbt_preds))

GBT Correct Accuracy
0.9702906071613907

In [0]:
dtc_model.featureImportances

Out[29]: SparseVector(4, {1: 0.0013, 2: 0.9551, 3: 0.0436})

In [0]:
rfc_model.featureImportances

Out[30]: SparseVector(4, {0: 0.0348, 1: 0.0246, 2: 0.9076, 3: 0.0329})

In [0]:
gbt_model.featureImportances

Out[31]: SparseVector(4, {0: 0.0254, 1: 0.0013, 2: 0.9032, 3: 0.0702})

In [0]:
rfc = RandomForestClassifier(numTrees=150,labelCol='Spoiled',featuresCol='features')
rfc_model = rfc.fit(train_df)
rfc_preds = rfc_model.transform(test_df)


In [0]:
print('RFC Accuracy')
print(binary_eval.evaluate(rfc_preds))

RFC Accuracy
0.9671769590036337

In [0]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

acc_eval = MulticlassClassificationEvaluator(labelCol='Spoiled',metricName='accuracy')

rfc_acc = acc_eval.evaluate(rfc_preds)

rfc_acc

Out[28]: 0.9777777777777777